In [1]:
import os

In [2]:
%pwd

'e:\\Data_Science_Project\\TalentGuard\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Data_Science_Project\\TalentGuard'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [7]:
from TalentGuard.constants import *
from TalentGuard.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
import numpy as np
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        f1 = f1_score(actual, pred)
        roc_auc = roc_auc_score(actual, pred)
        return accuracy, precision, recall, f1, roc_auc

    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]].values.ravel() 

        predicted_qualities = model.predict(test_x)

        
        accuracy, precision, recall, f1, roc_auc = self.eval_metrics(test_y, predicted_qualities)

        scores = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "roc_auc_score": roc_auc
        }
        save_json(path=Path(self.config.metric_file_name), data=scores)



In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2025-01-11 15:17:21,252: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-11 15:17:21,259: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-11 15:17:21,264: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-11 15:17:21,267: INFO: common: created directory at: artifacts]
[2025-01-11 15:17:21,270: INFO: common: created directory at: artifacts/model_evaluation]
[2025-01-11 15:17:23,803: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
